In [1]:
import pandas as pd

In [2]:
measurements = "resources/hawaii_measurements.csv"
stations = "resources/hawaii_stations.csv"
measurements_df = pd.read_csv(measurements)
stations_df = pd.read_csv(stations)
print(measurements_df.columns)
print(stations_df.columns)
print("*****************************************")
print(measurements_df.shape)
print("------------------------")
print(stations_df.shape)
print("*****************************************")
print(measurements_df.head())
print("------------------------")
print(stations_df.head())
print("*****************************************")
print("*****************************************")

## Cleaning Null Values
measurements_df=measurements_df.dropna(axis=0, how='any')
stations_df=stations_df.dropna(axis=0, how='any')

print(measurements_df.shape)
print(stations_df.shape)

## Save to csv
clean_measurements = measurements_df.to_csv("resources/clean_measurements.csv") #Overwrite old file
clean_stations = stations_df.to_csv("resources/clean_stations.csv")

measurements_df.head()

Index(['station', 'date', 'prcp', 'tobs'], dtype='object')
Index(['station', 'name', 'latitude', 'longitude', 'elevation'], dtype='object')
*****************************************
(19550, 4)
------------------------
(9, 5)
*****************************************
       station        date  prcp  tobs
0  USC00519397  2010-01-01  0.08    65
1  USC00519397  2010-01-02  0.00    63
2  USC00519397  2010-01-03  0.00    74
3  USC00519397  2010-01-04  0.00    76
4  USC00519397  2010-01-06   NaN    73
------------------------
       station                                    name  latitude  longitude  \
0  USC00519397                    WAIKIKI 717.2, HI US   21.2716  -157.8168   
1  USC00513117                    KANEOHE 838.1, HI US   21.4234  -157.8015   
2  USC00514830  KUALOA RANCH HEADQUARTERS 886.9, HI US   21.5213  -157.8374   
3  USC00517948                       PEARL CITY, HI US   21.3934  -157.9751   
4  USC00518838              UPPER WAHIAWA 874.3, HI US   21.4992  -158.0111   


,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
5,USC00519397,2010-01-07,0.06,70


In [3]:
# Dependencies and boilerplate
import sqlalchemy
from sqlalchemy import Column, Float, Integer, String, Text, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, inspect, func, MetaData
from datetime import datetime

# from sqlalchemy import *
# metadata = MetaData()

In [4]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
conn = engine.connect()

In [6]:
Base = declarative_base()

In [7]:
# https://stackoverflow.com/questions/30344237/error-sqlite-datetime-type-only-accepts-python-datetime-and-date-objects-a

class Measurement(Base):
    __tablename__ = "measurement"
    
    id = Column(Integer, primary_key=True)
    date = Column(Text)
    prcp = Column(Float) 
    station = Column(Text)
    tobs = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    
    
class Station(Base):
    __tablename__ = "station"
    
    id = Column(Integer, primary_key=True)
    elevation = Column(Float)
    latitude = Column(Float)
    longitude = Column(Float)        
    name = Column(Text)
    station = Column(Text)

    def __repr__(self):
        return f"id={self.id}, name={self.name}"

# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
Base.metadata.tables # Right now, this table only exists in python and not in the actual database

immutabledict({'measurement': Table('measurement', MetaData(bind=None), Column('id', Integer(), table=<measurement>, primary_key=True, nullable=False), Column('date', Text(), table=<measurement>), Column('prcp', Float(), table=<measurement>), Column('station', Text(), table=<measurement>), Column('tobs', Integer(), table=<measurement>), schema=None), 'station': Table('station', MetaData(bind=None), Column('id', Integer(), table=<station>, primary_key=True, nullable=False), Column('elevation', Float(), table=<station>), Column('latitude', Float(), table=<station>), Column('longitude', Float(), table=<station>), Column('name', Text(), table=<station>), Column('station', Text(), table=<station>), schema=None)})

In [8]:
Base.metadata.create_all(engine) # Create the 2 tables within the database

In [9]:
# csv to df to dict.
cm_df = "resources/clean_measurements.csv"
cs_df = "resources/clean_stations.csv"
cm_df = pd.read_csv(cm_df)
cs_df = pd.read_csv(cs_df)

cm_df=cm_df.drop(['Unnamed: 0'], axis=1).reset_index(drop=True)
cs_df=cs_df.drop(['Unnamed: 0'], axis=1).reset_index(drop=True)

cm_dic = cm_df.to_dict(orient='records')
cs_dic = cs_df.to_dict(orient='records')

In [10]:
# Use MetaData from SQLAlchemy to reflect the tables
# metadata = MetaData(bind=engine, reflect=True)
metadata = MetaData(bind=engine)
metadata.reflect()

In [11]:
# Populate SQLITE Table for Measurement_df
# m_table = Measurement.__table__
m_table = sqlalchemy.Table('measurement', metadata, autoload=True)
conn.execute(m_table.delete())
conn.execute(m_table.insert(), cm_dic)

# Populate SQLITE Table for stations_df
# s_table = Station.__table__
s_table = sqlalchemy.Table('station', metadata, autoload=True)
conn.execute(s_table.delete())
conn.execute(s_table.insert(), cs_dic)

In [12]:
# with engine.begin() as cn:
#    cn.execute(m_table.delete())
#    cn.execute(m_table.insert(), cm_dic)
    
conn.execute("select * from measurement").fetchall()

[(1, '2010-01-01', 0.08, 'USC00519397', 65),
 (2, '2010-01-02', 0.0, 'USC00519397', 63),
 (3, '2010-01-03', 0.0, 'USC00519397', 74),
 (4, '2010-01-04', 0.0, 'USC00519397', 76),
 (5, '2010-01-07', 0.06, 'USC00519397', 70),
 (6, '2010-01-08', 0.0, 'USC00519397', 64),
 (7, '2010-01-09', 0.0, 'USC00519397', 68),
 (8, '2010-01-10', 0.0, 'USC00519397', 73),
 (9, '2010-01-11', 0.01, 'USC00519397', 64),
 (10, '2010-01-12', 0.0, 'USC00519397', 61),
 (11, '2010-01-14', 0.0, 'USC00519397', 66),
 (12, '2010-01-15', 0.0, 'USC00519397', 65),
 (13, '2010-01-16', 0.0, 'USC00519397', 68),
 (14, '2010-01-17', 0.0, 'USC00519397', 64),
 (15, '2010-01-18', 0.0, 'USC00519397', 72),
 (16, '2010-01-19', 0.0, 'USC00519397', 66),
 (17, '2010-01-20', 0.0, 'USC00519397', 66),
 (18, '2010-01-21', 0.0, 'USC00519397', 69),
 (19, '2010-01-22', 0.0, 'USC00519397', 67),
 (20, '2010-01-23', 0.0, 'USC00519397', 67),
 (21, '2010-01-24', 0.01, 'USC00519397', 71),
 (22, '2010-01-25', 0.0, 'USC00519397', 67),
 (23, '2010-01-

In [13]:
conn.execute("select * from station limit 5").fetchall()

[(1, 3.0, 21.2716, -157.8168, 'WAIKIKI 717.2, HI US', 'USC00519397'),
 (2, 14.6, 21.4234, -157.8015, 'KANEOHE 838.1, HI US', 'USC00513117'),
 (3, 7.0, 21.5213, -157.8374, 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'USC00514830'),
 (4, 11.9, 21.3934, -157.9751, 'PEARL CITY, HI US', 'USC00517948'),
 (5, 306.6, 21.4992, -158.0111, 'UPPER WAHIAWA 874.3, HI US', 'USC00518838')]

In [14]:
#         # Ref: https://stackoverflow.com/questions/41900593/csv-into-sqlite-table-python

# import sqlite3
# df = pd.read_csv('resources/clean_stations.csv')
# con = sqlite3.connect("dummy.db")
# df.to_sql("Stations_test", con)

# con.close()